# preprocessing data for each region and auto tunning 



In [1]:
#%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import regularizers
import keras_tuner as kt
import pandas as pd
import numpy as np
import datetime
import os
import sys
import copy
import time
#rm -rf ./logs/

print('python: %s' % sys.version)
np.random.seed(2206)
print('tf version: %s' % tf.__version__) 
print('setting log device placement for both gpu and cpu not supported rn')
# from tensorflow.python.eager import context  
# _ = tf.Variable([1])  
# context._context = None 
# context._create_context()  
# tf.debugging.set_log_device_placement(True)
GPU_devices = tf.config.list_physical_devices('GPU')
# GPU_device = '/physical_device:GPU:0'
GPU_device = '/GPU:0'
print(GPU_device)
# CPU_devices = tf.config.list_physical_devices('CPU')
# CPU_device = '/physical_device:CPU:0'
# print('cpu : ' + '\n' + str(CPU_devices))
# print('gpu ' + '\n' + str(GPU_devices))
# tf.config.experimental.set_memory_growth(GPU_device, True)


# print('\n'*6 + '## configure threads')
# print('number of threads between independent operations: %s' % tf.config.threading.get_inter_op_parallelism_threads())
# print('number of threads within individual operations parallelism: %s' % tf.config.threading.get_intra_op_parallelism_threads())
# import multiprocessing
# numCores=multiprocessing.cpu_count()
# print(numCores,'cores available')
# tf.config.threading.set_inter_op_parallelism_threads(12) 
# tf.config.threading.set_intra_op_parallelism_threads(12)
# tf.config.set_soft_device_placement(True) # tf 2.7
# tf.compat.v1.config.set_soft_device_placement(True)
# print(tf.config.list_physical_devices(device_type=None))
# print('number of threads between independent operations: %s' % tf.config.threading.get_inter_op_parallelism_threads())
# print('number of threads within individual operations parallelism: %s' % tf.config.threading.get_intra_op_parallelism_threads())
# print('nvidia-smi')
#!nvidia-smi    

# from tensorflow.python.client import device_lib
# custom_objects={"GlorotUniform": tf.keras.initializers.glorot_uniform}
# print("Device Query:")
# print(device_lib.list_local_devices())

# print('\n'*6 + '## testing placement')
# _ = tf.Variable([1])  

# with tf.device('/device:GPU:0'):
#     _ = tf.Variable([1])

# with tf.device("/device:CPU:0"):
#     _ = tf.Variable([1])
    
# for muliple gpu workflow
# print(tf.config.list_physical_devices(device_type=None))
from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
#set_session(InteractiveSession(config=config))
# session = InteractiveSession(config=config)
# strategy = tf.distribute.MirroredStrategy() 
# print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

2021-11-19 07:19:57.660346: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


python: 3.8.10 (default, Sep 28 2021, 16:10:42) 
[GCC 9.3.0]
tf version: 2.5.0
setting log device placement for both gpu and cpu not supported rn
/GPU:0


2021-11-19 07:19:59.489500: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-19 07:19:59.530677: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-11-19 07:19:59.530716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-19 07:19:59.534256: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-19 07:19:59.534303: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-11-19 07:19:59.535461: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.

In [2]:
def print_time():
    print(time.asctime(time.localtime(time.time())))
    print(12*"#","\n")

print_time()

Fri Nov 19 07:19:59 2021
############ 



### Begin TMD

In [3]:
%%time
print(os.getcwd())
path2Data = '../data/training/'
Files = [file for file in sorted(os.listdir(path2Data)) if file.endswith('.csv')]

print(Files)
#file = Files[3]
file = 'EIC_aff_tmdaff.pkl'
print(file)

/home/share/TMD_Colab/validation/training
['EIC_aff_collinearaff.csv', 'EIC_aff_currentaff.csv', 'EIC_aff_targetaff.csv', 'EIC_aff_tmdaff.csv', 'bigData.csv']
EIC_aff_tmdaff.pkl
CPU times: user 0 ns, sys: 1.05 ms, total: 1.05 ms
Wall time: 753 µs


In [4]:
%%time
train_features = np.load(path2Data+file, allow_pickle=True)
#train_features = train_features.drop(columns=['hadron'])
train_labels = train_features.pop('tmdaff')
print(train_features)
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

                  pT           Q2         x    z  R0max  R1max  R2max
0           0.100000     1.000000  0.000062  0.1   0.05   0.05   0.05
1           0.193611     1.000000  0.000062  0.1   0.05   0.05   0.05
2           0.374852     1.000000  0.000062  0.1   0.05   0.05   0.05
3           0.725756     1.000000  0.000062  0.1   0.05   0.05   0.05
4           1.405143     1.000000  0.000062  0.1   0.05   0.05   0.05
...              ...          ...       ...  ...    ...    ...    ...
102297595   2.008343  6536.401637  0.545559  0.9   1.20   1.20   1.20
102297596   3.659362  6536.401637  0.545559  0.9   1.20   1.20   1.20
102297597   6.667653  6536.401637  0.545559  0.9   1.20   1.20   1.20
102297598  12.149000  6536.401637  0.545559  0.9   1.20   1.20   1.20
102297599  22.136456  6536.401637  0.545559  0.9   1.20   1.20   1.20

[102297600 rows x 7 columns]
0            0.0
1            0.0
2            0.0
3            0.0
4            0.0
            ... 
102297595    1.0
102297596  

2021-11-19 07:20:03.743890: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-19 07:20:03.748803: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2021-11-19 07:20:03.751148: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2021-11-19 07:20:03.751202: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-19 07:20:04.590253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExe

<class 'tensorflow.python.framework.ops.EagerTensor'>
CPU times: user 5.17 s, sys: 12.3 s, total: 17.4 s
Wall time: 17.5 s


In [5]:
%%time
input_shape = train_features.shape[1:]
print(input_shape)
classes = 1
model = keras.models.load_model('./models/tmd_norm_layer')
normalizer = model.layers.pop(0)
# To be used as a normalizing input layer in DNN
# normalizer = preprocessing.Normalization(axis=-1)
# normalizer.adapt(np.array(train_features))

# model = keras.Sequential([normalizer])

# model.save('./models/tmd_norm_layer')

(7,)
CPU times: user 244 ms, sys: 513 ms, total: 757 ms
Wall time: 762 ms


In [6]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) # 15 trials
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) # 21 trials
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) # m=315 experiments

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='sigmoid'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_TMD_sigmoid_V6.1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='tmd_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 87 Complete [00h 21m 45s]
val_loss: 0.00010421664774185047

Best val_loss So Far: 0.00010421664774185047
Total elapsed time: 06h 37m 39s

Search: Running Trial #88

Hyperparameter    |Value             |Best Value So Far 
units             |832               |576               
units2            |400               |160               
learning_rate     |0.0001            |0.0001            
tuner/epochs      |30                |30                
tuner/initial_e...|0                 |0                 
tuner/bracket     |0                 |0                 
tuner/round       |0                 |0                 

Epoch 1/30
11060/11060 [==============================] - 52s 4ms/step - loss: 0.0133 - val_loss: 0.0037
Epoch 2/30
11060/11060 [==============================] - 47s 4ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 3/30
11060/11060 [==============================] - 46s 4ms/step - loss: 8.5428e-04 - val_loss: 0.0011
Epoch 4/30
11060/11060 [=============================

In [ ]:
tuner.results_summary()

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='linear'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_TMD_linear_V6.2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='tmd_auto_tune_final_run')

with tf.device(GPU_device):
    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
%%time
tuner.results_summary()

### Begin Collinear

In [ ]:
%%time

file = 'EIC_aff_collinearaff.pkl'
print('\nYou have selected:',file)

In [ ]:
%%time
train_features = np.load(path2Data+file, allow_pickle=True)

train_labels = train_features.pop('collinearaff')
print(train_features.head())
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
%%time

input_shape = train_features.shape[1:]
print(input_shape)
classes = 1
model = keras.models.load_model('./models/collinear_norm_layer') 
normalizer = model.layers.pop(0)

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='sigmoid'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_COLLINEAR_sigmoid_V6.1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='collinear_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
tuner.results_summary()

In [ ]:
tuner.results_summary()

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='linear'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_COLLINEAR_linear_V6.2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='collinear_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
%%time
tuner.results_summary()

### Begin Current

In [ ]:
%%time
file = 'EIC_aff_currentaff.pkl'
print('\nYou have selected:',file)

In [ ]:
%%time
train_features = np.load(path2Data+file, allow_pickle=True)
#train_features = train_features.drop(columns=['hadron'])
train_labels = train_features.pop('currentaff')
print(train_features)
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
%%time
input_shape = train_features.shape[1:]
print(input_shape)
classes = 1

# To be used as a normalizing input layer in DNN
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

model = keras.Sequential([normalizer])

model.save('./models/current_norm_layer')

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='sigmoid'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_CURRENT_sigmoid_V6.1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='current_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
tuner.results_summary()

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='linear'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_CURRENT_linear_V6.2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='current_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                 callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
%%time
tuner.results_summary()

### Begin Target

In [ ]:
%%time
file = 'EIC_aff_targetaff.pkl'
print('\nYou have selected:',file)

In [ ]:
%%time
train_features = np.load(path2Data+file, allow_pickle=True)
#train_features = train_features.drop(columns=['hadron'])
train_labels = train_features.pop('targetaff')
print(train_features)
print(train_labels)
print(type(train_features))
train_features = tf.convert_to_tensor(train_features)
print(type(train_features))
train_labels = tf.convert_to_tensor(train_labels)

In [ ]:
%%time
input_shape = train_features.shape[1:]
print(input_shape)
classes = 1

# To be used as a normalizing input layer in DNN
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

model = keras.Sequential([normalizer])

model.save('./models/target_norm_layer')

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64) 
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48)
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='sigmoid'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_TARGET_sigmoid_V6.1/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='target_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
tuner.results_summary()

In [ ]:
def model_builder(hp):

    model = keras.Sequential([normalizer])

        
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # 2nd hidden layer
    hp_units2 = hp.Int('units2', min_value=16, max_value=1024, step=48) 
    model.add(keras.layers.Dense(units=hp_units2, activation='relu')) 

    # add output layer
    # activation = hp.Choice(name = 'activation', values = ['softmax', 'sigmoid', 'relu', 'linear'], ordered = False)
    model.add(keras.layers.Dense(1, activation='linear'))# activation='relu'),# kernel_initializer='he_uniform')#, activation="linear")
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error')


    return model

In [ ]:
%%time

log_dir = "log/autotune_TARGET_linear_V6.2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=30,
                     factor=3,
                     directory=log_dir,
                     project_name='target_auto_tune_final_run')

# tuner = kt.RandomSearch(hypermodel=model_builder,
#                         objective='val_loss',
#                         seed=22,
#                         max_trials=100,
#                         executions_per_trial=1,
#                         directory=log_dir+'/random_search',
#                         project_name='tmd_auto_tune_first_layer_complete_dataset')



with tf.device(GPU_device):

    tuner.search(train_features, train_labels, epochs=30, validation_split=0.2, shuffle = True, batch_size=7400,
                callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
%%time
tuner.results_summary()

In [ ]:
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)